In [1]:
import pandas as pd
import mysql.connector

In [5]:
connection = mysql.connector.connect(
    host= 'localhost',         
    user= 'root',             
    password= '123456', 
    database = 'day_49'     
)
cursor = connection.cursor()

In [3]:
cursor.execute("""
CREATE TABLE customers (
    id INTEGER PRIMARY KEY AUTO_INCREMENT,
    name VARCHAR(200) NOT NULL, 
    city VARCHAR(200) NOT NULL
);
""")

In [4]:
cursor.execute("""
INSERT INTO customers (id, name, city) VALUES
(1, 'Alice', 'Hanoi'),
(2, 'Bob', 'Ho Chi Minh'),
(3, 'Charlie', 'Hanoi');
""")

connection.commit()

In [5]:
cursor.execute("""
CREATE TABLE orders (
    id INTEGER PRIMARY KEY AUTO_INCREMENT,
    customer_id INTEGER NOT NULL,
    order_date DATE NOT NULL,
    total INTEGER NOT NULL,
    FOREIGN KEY (customer_id) REFERENCES customers(id)
);
""")

In [6]:
cursor.execute("""
INSERT INTO orders (id, customer_id, order_date, total) VALUES
(1, 1, '2024-01-10', 500),
(2, 1, '2024-03-01', 800),
(3, 2, '2024-02-20', 1200),
(4, 3, '2024-01-15', 200);
""")

connection.commit()

In [7]:
cursor.execute("""
CREATE TABLE products (
    id INTEGER PRIMARY KEY AUTO_INCREMENT,
    name VARCHAR(200) NOT NULL,
    price INTEGER NOT NULL
);
""")

In [8]:
cursor.execute("""
INSERT INTO products (id, name, price) VALUES
(1, 'Laptop', 1500),
(2, 'Mouse', 50),
(3, 'Keyboard', 100),
(4, 'Monitor', 300);
""")

connection.commit()

In [9]:
cursor.execute("""
CREATE TABLE order_items (
    id INTEGER PRIMARY KEY AUTO_INCREMENT,
    order_id INTEGER NOT NULL,
    product_id INTEGER NOT NULL,
    quantity INTEGER NOT NULL,
    FOREIGN KEY (order_id) REFERENCES orders(id),
    FOREIGN KEY (product_id) REFERENCES products(id)
);
""")

In [10]:
cursor.execute("""
INSERT INTO order_items (id, order_id, product_id, quantity) VALUES
(1, 1, 2, 2),
(2, 1, 3, 1),
(3, 2, 1, 1),
(4, 2, 4, 1),
(5, 3, 1, 1);
""")

connection.commit()

In [11]:
cursor.execute("""
CREATE TABLE employees (
    id INTEGER PRIMARY KEY AUTO_INCREMENT,
    name VARCHAR(200) NOT NULL,
    department VARCHAR(200) NOT NULL
);
""")

In [12]:
cursor.execute("""
INSERT INTO employees (id, name, department) VALUES
(1, 'David', 'Sales'),
(2, 'Emma', 'Support'),
(3, 'Frank', 'Sales');
""")

connection.commit()

In [13]:
cursor.execute("""
CREATE TABLE order_assignments (
    id INTEGER PRIMARY KEY AUTO_INCREMENT,
    order_id INTEGER NOT NULL,
    employee_id INTEGER NOT NULL,
    FOREIGN KEY (order_id) REFERENCES orders(id),
    FOREIGN KEY (employee_id) REFERENCES employees(id)
);
""")

In [14]:
cursor.execute("""
INSERT INTO order_assignments (id, order_id, employee_id) VALUES
(1, 1, 1),
(2, 2, 3),
(3, 3, 2);
""")

connection.commit()

### Bài 1

In [16]:
cursor.execute("DROP PROCEDURE IF EXISTS TotalOrderAmount")
cursor.execute("""
CREATE PROCEDURE TotalOrderAmount(IN customer_id INT, OUT total_order_amount INT)
BEGIN
    SELECT COUNT(o.id) INTO total_order_amount
    FROM orders o
    WHERE o.customer_id = customer_id;
END;
""")
connection.commit()
print("Stored procedure created.")

Stored procedure created.


In [20]:
customer_id = 1  
order_count = 0  

results = cursor.callproc('TotalOrderAmount', (customer_id, order_count))

print(f"Tổng số đơn hàng của khách hàng có id {customer_id} là:", results[1])

Tổng số đơn hàng của khách hàng có id 1 là: 2


### Bài 2

In [23]:
cursor.execute("DROP PROCEDURE IF EXISTS EmployeeName")
cursor.execute("""
CREATE PROCEDURE EmployeeName(IN employee_id INT, OUT employee_name VARCHAR(200))
BEGIN
    SELECT e.name INTO employee_name
    FROM employees e
    WHERE e.id = employee_id;
END;
""")
connection.commit()
print("Stored procedure created.")

Stored procedure created.


In [24]:
employee_id = 1  
employee_name = ''  

results = cursor.callproc('EmployeeName', (employee_id, employee_name))

print(f"Tên nhân viên có id {employee_id} là:", results[1])

Tên nhân viên có id 1 là: David


### Bài 3

In [25]:
cursor.execute("DROP PROCEDURE IF EXISTS UpdateCityName")
cursor.execute("""
CREATE PROCEDURE UpdateCityName(IN customer_id INT, IN new_city VARCHAR(200))
BEGIN
    UPDATE customers
    SET city = new_city
    WHERE id = customer_id;
END;
""")
connection.commit()
print("Stored procedure created.")

Stored procedure created.


In [26]:
customer_id = 1  
new_city = 'HaiPhong'  

results = cursor.callproc('UpdateCityName', (customer_id, new_city))

print(f"Đã cập nhật thành phố của khách hàng có id {customer_id} là:", results[1])

Đã cập nhật thành phố của khách hàng có id 1 là: HaiPhong


In [28]:
query = ("""
SELECT id, name, city
 FROM customers 
WHERE id = 1
""")

data_3 = pd.read_sql_query(query, connection)
print(data_3)

   id   name      city
0   1  Alice  HaiPhong


C:\Users\Apollonir\AppData\Local\Temp\ipykernel_12444\4286663439.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_3 = pd.read_sql_query(query, connection)


### Bài 4

In [46]:
cursor.execute("DROP PROCEDURE IF EXISTS ProductHavePriceBigger")
cursor.execute("""
CREATE PROCEDURE ProductHavePriceBigger(IN min_price INT)
BEGIN
    SELECT p.name, p.price
    FROM products p
    WHERE p.price >= min_price;
END;
""")
connection.commit()
print("Stored procedure created.")

Stored procedure created.


In [47]:
min_price = 300 

cursor.callproc('ProductHavePriceBigger', [min_price])

for result in cursor.stored_results():
    rows = result.fetchall()
    for row in rows:
        print(row)

('Laptop', 1500)
('Monitor', 300)


### Bài 5

In [55]:
cursor.execute("DROP PROCEDURE IF EXISTS DeleteOrder")
cursor.execute("""
CREATE PROCEDURE DeleteOrder(IN min_total INT)
BEGIN
    DELETE FROM order_items 
    WHERE order_id IN (SELECT id FROM orders WHERE total < min_total);

    DELETE FROM order_assignments
    WHERE order_id IN (SELECT id FROM orders WHERE total < min_total);
    
    DELETE FROM orders
    WHERE total < min_total;
END;
""")
connection.commit()
print("Stored procedure created.")

Stored procedure created.


In [57]:
min_total = 900 

cursor.callproc('DeleteOrder', [min_total])

print(f"Đã xóa các đơn hàng có tổng giá trị nhỏ hơn {min_total}")

Đã xóa các đơn hàng có tổng giá trị nhỏ hơn 900


In [58]:
query = ("""
SELECT id, order_date, total
 FROM orders 
""")

data_5 = pd.read_sql_query(query, connection)
print(data_5)

   id  order_date  total
0   3  2024-02-20   1200


C:\Users\Apollonir\AppData\Local\Temp\ipykernel_12444\1336282475.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_5 = pd.read_sql_query(query, connection)
